# 0) Install and import all the needed dependencies

You need to start by installing three packages: mediapipe, opencv and scikit.


In [ ]:
#Once you have run this first block once you will not need to run it again.
#This just installs these packages onto your local machine.
!pip install mediapipe==0.10.9 opencv-python scikit-learn

# 0)  Make some detections 

Run this if you want to make sure you have dependencies installed correctly.

<!-- Hand Landmark Image Map <img src="https://i.imgur.com/8bForKY.png">-->
POSE GESTURE LANDMARKS GUIDE
<img src="https://i.imgur.com/AzKNp7A.png">

In [1]:
import mediapipe as mp #Imports mediapipe
import cv2 #Imports open cv
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #precents copying data but still allows use of image for rendering
        
        # Make Detections
        results = holistic.process(image)
                
        # Recolor image back to BGR for rendering
        image.flags.writeable = True #precents copying data but still allows use of image for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand - Draw the right hand keypoints
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand - Draw the left hand keypoints
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections - Draw the body keypoints as pictured below
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1717120838.505623       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 1) CAPTURE LANDMARK DATA FOR A POSE AND SAVE IT TO A SPREADSHEET

This creates a spreadsheet with a number of columns equal to the number of keypoints in your pose (33 total - it ignores the hands and face in the holistic model).

In [2]:
import csv
import os
import numpy as np
num_coords = len(results.pose_landmarks.landmark)
num_coords
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]
with open('acsm.csv', mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
class_name = input("Enter the name for the pose : ")

Enter the name for the pose : Right Tandem


In [12]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #prevents copying data but still allows use of image for rendering
        
        # Make Detections
        results = holistic.process(image)        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw the Right hand - Purely for show. The model doesn't use these in fitting an ML model
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Draw the Left Hand - Purely for show. The model doesn't use these in fitting an ML model
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections - This is where the magic is! These are the points you'll use
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Export coordinates
        try:
            #Extract pose and face landmarks and put into numpy arrays
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            #print(pose_row) #Uncomment to output the poserow data to the screen

            #concatenate rows
            row = pose_row
            
            #append the class name to the row in the zeroth column
            row.insert(0, class_name)
            
            #export to csv
            with open('acsm.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            print("NERP MAGERP")
            pass
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1717121356.838025       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro


NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP
NERP MAGERP


## 2) READ SPREADSHEET INTO A DATAFRAME

Takes the spreadsheet from section 1 and reads it into a dataframe

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split # allows you to make a training partition and a testing partition with your data
df = pd.read_csv('acsm.csv') # reads in your coords.csv file into a dataframe
df.head()
len(df)

6311

## 3) CREATE AN ML MODEL

This will use the dataframe created in section 3 to generate multiple machine learning models (Logistic Regression, Ridge Regression, Random Forest and Gradient Descent). It will analyze these models and choose the best performing one and output into a file using JOBLIB for later use on live classification.

## Create a model to deploy to live classification

This next block will take the BEST performing ML model from above and create a classification model that can be applied to real-time processing using the same blazpose pose recognition algorithm that you used to collect the data.

In [15]:
import joblib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

# Assuming you have a DataFrame `df` with your data and the 'pose' column as the target
x = df.drop('class', axis=1)
y = df['class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234, stratify=y)

# Define the models
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'Ridge Classifier': RidgeClassifier(),
    'Random Forest': RandomForestClassifier(random_state=1234),
    'Gradient Boosting': GradientBoostingClassifier(random_state=1234)
}

# Create pipelines for each model
pipelines = {name: make_pipeline(StandardScaler(), model) for name, model in models.items()}

# Train and evaluate each model using cross-validation
cv_scores = {}
for name, pipeline in pipelines.items():
    cv_scores[name] = cross_val_score(pipeline, X_train, y_train, cv=5)
    print(f'{name} CV Score: {cv_scores[name].mean():.4f}')

# Find the best model based on cross-validation score
best_model_name = max(cv_scores, key=lambda name: cv_scores[name].mean())
print(f'Best Model: {best_model_name} with CV Score: {cv_scores[best_model_name].mean():.4f}')

# Fit the best model to the entire training set and evaluate on the test set
best_model = pipelines[best_model_name]
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Print evaluation metrics for the test set
print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

# Save the trained model and scaler to a file
joblib_file_model = "best_tuned_model.joblib"
joblib.dump(best_model, joblib_file_model)
print(f"Tuned model saved to {joblib_file_model}")

Logistic Regression CV Score: 0.9934
Ridge Classifier CV Score: 0.9842
Random Forest CV Score: 0.9968
Gradient Boosting CV Score: 0.9957
Best Model: Random Forest with CV Score: 0.9968
Classification Report:
              precision    recall  f1-score   support

  Double Leg       1.00      1.00      1.00       475
 Left Single       1.00      1.00      1.00       306
 Left Tandem       1.00      1.00      1.00       425
Right Single       1.00      1.00      1.00       348
Right Tandem       1.00      0.99      1.00       340

    accuracy                           1.00      1894
   macro avg       1.00      1.00      1.00      1894
weighted avg       1.00      1.00      1.00      1894

Confusion Matrix:
[[475   0   0   0   0]
 [  0 306   0   0   0]
 [  0   0 425   0   0]
 [  0   0   0 348   0]
 [  0   1   1   0 338]]
Tuned model saved to best_tuned_model.joblib


## 4) LIVE POSE CLASSIFICATION

This will load the joblib model created above and apply it to a live camera feed for live classification.

In [16]:
import cv2
import mediapipe as mp
import numpy as np
import joblib

# Load the saved model
best_tuned_model = joblib.load("best_tuned_model.joblib")

# Assuming you have a list of class names
class_names = best_tuned_model.classes_

# Initialize MediaPipe BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Extract keypoints
        keypoints = []
        for landmark in results.pose_landmarks.landmark:
            keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

        # Convert to numpy array and reshape
        keypoints = np.array(keypoints).reshape(1, -1)

        # Predict the pose and get the probabilities
        pose_probabilities = best_tuned_model.predict_proba(keypoints)[0]

        # Set the threshold to 50%
        threshold = 0.5

        # Prepare text for the detected pose
        font_scale_detected = 1.5
        font_thickness_detected = 2
        font = cv2.FONT_HERSHEY_SIMPLEX
        x, y = 10, 50

        detected_pose = np.argmax(pose_probabilities)
        detected_prob = pose_probabilities[detected_pose]

        if detected_prob >= threshold:
            detected_text = f'Detected Pose: {class_names[detected_pose]} ({detected_prob:.2f})'
        else:
            detected_text = 'No pose detected'

        # Draw background rectangle for the detected pose text
        (text_width, text_height), baseline = cv2.getTextSize(detected_text, font, font_scale_detected, font_thickness_detected)
        rect_x1, rect_y1 = x, y - text_height - baseline
        rect_x2, rect_y2 = x + text_width, y + baseline
        cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)

        # Display the detected pose text
        cv2.putText(frame, detected_text, (x, y), font, font_scale_detected, (255, 255, 255), font_thickness_detected, cv2.LINE_AA)

        # Prepare text for each class
        font_scale_table = 1
        font_thickness_table = 2
        table_start_y = y + text_height + 20  # Start position for the table
        line_height = 50  # Height between each line

        for i, class_name in enumerate(class_names):
            prob = pose_probabilities[i]
            text = f'{class_name}: {prob:.0%}'

            # Draw background rectangle for text
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale_table, font_thickness_table)
            rect_x1, rect_y1 = x, table_start_y + i * line_height - text_height - baseline
            rect_x2, rect_y2 = x + text_width, table_start_y + i * line_height + baseline
            cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)

            # Display text
            cv2.putText(frame, text, (x, table_start_y + i * line_height), font, font_scale_table, (255, 255, 255), font_thickness_table, cv2.LINE_AA)

        # Draw the pose landmarks on the frame
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the frame
    cv2.imshow('Pose Recognition', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1717121697.416555       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, bu

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

...

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/ebubar/anaconda3/lib/python3.11/site-packag

## Code below Deprecated on 5/30/24

In [ ]:
list(pipelines.values())[0]

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

### 3.3) Evaluate and Serialize (i.e. "saving") model

In [ ]:
from sklearn.metrics import accuracy_score #Accuracy metric (could use precision, recall, r1 score, ausc)
import pickle #Package allowing saving data to disk

In [ ]:
#This code is looping through your models and saving their prediction accuracy score.  yhat is the typical variable used for a "score"
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print (algo, accuracy_score(y_test,yhat))

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
y_test

In [ ]:
with open('body_language.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

# 4) Make Detections with Model

In [ ]:
with open('body_language.pkl','rb') as f:
    model = pickle.load(f)

In [ ]:
with open('body_language.pkl','rb') as f:
    model = pickle.load(f)

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #prevents copying data but still allows use of image for rendering
        
        # Make Detections
        results = holistic.process(image)
        class_name = "placeholder"
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Export coordinates
        try:
            #Extract pose and face landmarks and put into numpy arrays
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            #face = results.face_landmarks.landmark
            #face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            #concatenate rows
            row = pose_row#+face_row
            
            #append the class name to the row in the zeroth column
            row.insert(0, class_name)
        

            #Make Detection
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0] #probably that class is corre            
            
            #Grab coods
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                    ,[640,480]).astype(int))
                           
            cv2.rectangle(image,  
                         (coords[0], coords[1]+5), 
                         (coords[0]+len(body_language_class)*20, coords[1]-30),
                         (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
            #Get status box
            cv2.rectangle(image,(0,0), (250,60), (245,117,16),-1)
            
            #Display class
            cv2.putText(image, 'CLASS'
                       , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                       , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            #Display Probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        

cap.release()
cv2.destroyAllWindows()       

In [ ]:
print(body_language_class)